In [1]:
import sys
sys.path.append("/home/hyunho/sfda/")
import torch
import torch, gc

gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_source_model_path = "exp/deeplabv2_1024/best_model_4_accuracy=0.8350.pt"
import os
pretrained_source_model_path= os.path.join("/home/hyunho/sfda", pretrained_source_model_path)
print(f"device : {device}")

device : cuda


# Model debugging

In [2]:
import torch.nn as nn
from model.deeplabv2 import ResNetMulti, Bottleneck, ThresholdNet

class ResNetMultiBayes(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ResNetMultiBayes, self).__init__()
        self.resnet = ResNetMulti(block, layers, num_classes)
        self.thresholdnet = ThresholdNet(2048 ,256) # input size, output size
        self.avgpool = nn.AdaptiveAvgPool2d((19,1)) # class num?

    def forward(self, x,):
        output, feature = self.resnet(x, return_features=True)
        feature = self.avgpool(feature)
        mu, logvar = self.thresholdnet(torch.squeeze(feature, dim=-1).permute(0,2,1))

        # return torch.squeeze(feature,dim=-1)
        return output, mu, logvar

In [3]:
model = ResNetMultiBayes(Bottleneck, layers= [3, 4, 6, 3], num_classes=19).to(device)
model.resnet.load_state_dict(torch.load(pretrained_source_model_path, map_location=device, weights_only=True))

<All keys matched successfully>

In [4]:
import torch
from torch.utils.data import DataLoader
from dataset.cityscapes_loader import CityscapesDataset
from torchvision import transforms
from utils.data_augmentation import RandomResizedCropWithMask, RandomHorizontalFlip, rotate_image

cityscape_image_mean = (0.4422, 0.4379, 0.4246)
cityscape_image_std = (0.2572, 0.2516, 0.2467)
input_size = (720, 1280)

train_image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cityscape_image_mean, std=cityscape_image_std),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)]),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)])
])
valid_image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cityscape_image_mean, std=cityscape_image_std),
])
train_both_transforms = transforms.Compose([
    RandomHorizontalFlip(0.5),
    RandomResizedCropWithMask(input_size)
    ])

train_dataset = CityscapesDataset(
    images_dir="/home/hyunho/sfda/data/cityscapes_dataset/leftImg8bit/train",
    masks_dir="/home/hyunho/sfda/data/cityscapes_dataset/gtFine/train",
    image_transform=train_image_transforms,
    both_transform=train_both_transforms,
    debug=True
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=2, 
    shuffle=True, 
    pin_memory=True,
    num_workers=16
)
data = next(iter(train_loader))
img, label, name = data


/home/hyunho/miniconda3/envs/test/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [5]:
img = img.to(device)
output, mu, var  = model(img)

/home/hyunho/miniconda3/envs/test/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


# Loss Degbgging

In [6]:
from utils.loss import kld_loss
loss, threshold = kld_loss(mu, var)

In [7]:
print("loss : ", loss.item())
print("threshold : ", threshold)

loss :  9412.529296875
threshold :  tensor([ 0.2881,  0.0449,  0.0340,  0.1194,  0.0490,  0.0780,  0.1539,  0.0614,
         0.0861,  0.0945,  0.0566,  0.0450,  0.0168,  0.0678,  0.0710,  0.0914,
        -0.0100,  0.0539,  0.1351], device='cuda:0', grad_fn=<MeanBackward1>)


In [9]:
import torch.nn.functional as F
from utils.loss import SoftDiceLoss, softmax_cross_entropy_with_logits

def calculate_pseudo_loss(teacher_output,
                          student_output,
                          threshold,
                          temp):

    stu_prob_output = torch.softmax(student_output, dim=1)
    tut_prob_output = torch.softmax(teacher_output, dim=1)
    max_prob, max_class = tut_prob_output.max(dim=1)
    threshold_map = threshold[max_class]
    pseudo_labels = max_class.clone()
    pseudo_labels[max_prob < threshold_map] = 255
    background_mask = (pseudo_labels != 255).long()

    pseudo_labels = pseudo_labels.float()

    # for unlabelled parts:
    if len(pseudo_labels.size()) == 3:
        (b, h, w) = pseudo_labels.size()
        vol = b*h*w
    else:
        raise NotImplementedError

    if pseudo_labels.sum() > 0.0005*vol:
        loss_unsup = 0.5*SoftDiceLoss()(stu_prob_output, pseudo_labels, background_mask)
        loss_unsup += 0.5*softmax_cross_entropy_with_logits(student_output, pseudo_labels)
    else:
        loss_unsup = torch.zeros(1).cuda()

    return {'loss': loss_unsup,
            'prob': stu_prob_output.mean()}

/home/hyunho/miniconda3/envs/test/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [1158,0,0], thread: [39,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [1158,0,0], thread: [40,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_172478911678

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:
prob_output = torch.softmax(output, dim=1)
max_prob, max_class = prob_output.max(dim=1)
threshold_map = threshold[max_class]
result = max_class.clone()

result[max_prob < threshold_map] = 255
background_mask = (result != 255).long()

In [12]:
stu_prob_output = torch.softmax(output, dim=1)
stu_prob_output.shape

torch.Size([2, 19, 720, 1280])